In [106]:
import pandas as pd
from tqdm import tqdm
import os
import nltk
from nltk.corpus import stopwords

In [107]:
brexit_file = pd.read_csv('./brexit_full_updated3.csv')

politicians = []

count_zero = 0

politicians_party ={}

for index, row in brexit_file.iterrows():
    full_name = row['full name']
    party = row['party']
    
    politicians.append(full_name)
    politicians_party[short_name] = party
        
print(len(politicians))
politicians = list(set(politicians))
print(len(politicians))

660
660


In [198]:
brexit_file['full name'] = brexit_file['full name'].apply(lambda x: x.strip())
brexit_file

,Constituency ID,full name,PANO,Constituency Name,Party abbreviation,conShare,conShare2,conShare3,bill161,leave,party_old,ref,ref_dummy,party,old_party,2015-2016,2015-2016 EU,2016-2017,2016-2017 EU
0,E14000530,Gerald Howarth,7,Aldershot,Con,50.59,73.403947,73.403947,1.0,1.0,1.0,0.578978,1,1.0,1.0,10984,773,3280,698
1,E14000531,Wendy Morton,8,Aldridge-Brownhills,Con,52.05,69.940876,69.940876,1.0,0.0,1.0,0.677963,1,0.0,0.0,6952,201,11025,662
2,E14000532,Graham Brady,9,Altrincham and Sale West,Con,52.99,66.503514,66.503514,1.0,1.0,1.0,0.385878,0,1.0,1.0,2852,0,628,0
3,E14000533,Nigel Mills,11,Amber Valley,Con,43.98,55.840528,55.840528,1.0,1.0,1.0,0.652991,1,1.0,1.0,4477,0,5798,104
4,E14000534,Nick Herbert,18,Arundel and South Downs,Con,60.79,84.442284,84.442284,NaN,0.0,1.0,0.497011,0,0.0,0.0,3577,317,2819,87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,W07000076,Wayne David,114,Caerphilly,Lab,16.59,27.223499,27.223499,1.0,0.0,0.0,0.551360,1,0.0,NaN,7053,454,3326,525
656,W07000077,Chris Evans,336,Islwyn,Lab,15.16,23.639482,23.639482,1.0,0.0,0.0,0.589399,1,0.0,NaN,4697,0,5222,337
657,W07000078,Alun Cairns,589,Vale Of Glamorgan,Con,46.02,58.527280,58.527280,1.0,0.0,1.0,0.525507,1,0.0,0.0,8127,0,12034,0
658,W07000079,Kevin Brennan,129,Cardiff West,Lab,25.15,38.221884,38.221884,0.0,0.0,0.0,0.438226,0,0.0,NaN,15296,71,7025,123


In [112]:
def preprocess(topic):
    ret = []
    topic = topic.strip()
    
    if '|' in topic:
        topics = topic.split('|')
        t = topics[0]
        t = t.strip()
        return t
        
    return topic

In [155]:
def contain_multi(topic):
    topic = topic.strip()
    return '|' in topic

In [174]:
df1 = pd.read_csv('2015_commons_cleaned.csv')
df2 = pd.read_csv('2016_commons_cleaned.csv')
df = pd.concat([df1, df2])
df = df.drop(df[df.Topic == 'admin'].index)
df['Topic'] = df['Topic'].apply(lambda x: preprocess(x))
df['Date'] = pd.to_datetime(df['Date'])
mask = (df['Date'] >= '2015-6-1') & (df['Date'] <= '2016-6-30')
df = df[mask]
df['match'] = df['Name'].apply(lambda x: x in politicians)
df['len'] = df['Speech-Stemmed-unigram_count'].apply(lambda x: len(str(x).split()))
df1 = pd.read_csv('2016_commons_cleaned.csv')
df2 = pd.read_csv('2017_commons_cleaned.csv')
df1 = pd.concat([df1, df2])
df1['Date'] = pd.to_datetime(df1['Date'])
mask = (df1['Date'] >= '2016-6-1') & (df1['Date'] <= '2017-6-30')
df1 = df1[mask]
df1 = df1.drop(df1[df1.Topic == 'admin'].index)
df1['Topic'] = df1['Topic'].apply(lambda x: preprocess(x))
df1['match'] = df1['Name'].apply(lambda x: x in politicians)
df1['len'] = df1['Speech-Stemmed-unigram_count'].apply(lambda x: len(str(x).split()))

In [199]:
names1 = df['Name'].unique()
names2 = brexit_file['full name'].unique()

names1_not_in_names2 = [n for n in names1 if n not in names2]
names2_not_in_names1 = [n for n in names2 if n not in names1]

In [200]:
brexit_file['full name'].unique()

array(['Gerald Howarth', 'Wendy Morton', 'Graham Brady', 'Nigel Mills',
       'Nick Herbert', 'Gloria De Piero', 'Damian Green', 'Angela Rayner',
       'David Lidington', 'Victoria Prentis', 'Margaret Hodge',
       'Dan Jarvis', 'Michael Dugher', 'John Woodcock', 'John Baron',
       'Maria Miller', 'John Mann', 'Ben Howlett', 'Tracy Brabin',
       'Jo Cox', 'Jane Ellison', 'Dominic Grieve', 'Bob Stewart',
       'Richard Fuller', 'Neil Coyle', 'Anne-Marie Trevelyan',
       'Rushanara Ali', 'Graham Stuart', 'Huw Merriman', 'David Evennett',
       'Frank Field', 'Gisela Stuart', 'Jack Dromey', 'Roger Godsiff',
       'Liam Byrne', 'Shabana Mahmood', 'Richard Burden',
       'Khalid Mahmood', 'Steve McCabe', 'Jess Phillips', 'Helen Goodman',
       'Kate Hollern', 'Graham Stringer', 'Paul Maynard',
       'Gordon Marsden', 'Dave Anderson', 'Ronnie Campbell', 'Nick Gibb',
       'Dennis Skinner', 'David Crausby', 'Yasmin Qureshi', 'Chris Green',
       'Peter Dowd', 'Matt Warman', '

In [201]:
names1_not_in_names2

['Mr Speaker',
 'Mr Deputy Speaker',
 'Madam Deputy Speaker',
 'Several hon Members',
 'Hon Members',
 'The Temporary Chair',
 'The Chairman',
 'The Second Deputy Chairman',
 'The Temporary Chairman',
 'The First Deputy Chairman']

In [181]:
print(len(names1_not_in_names2))
print(len(names2_not_in_names1))

11
22


In [191]:
df = df.loc[df['Name'].isin(names1_not_in_names2)]

In [196]:
df[['Name', 'len']].groupby(['Name'], as_index=False).count().to_csv('missing_politicians_speech_count.csv')

In [182]:
session_df = pd.read_csv('WordFreqYear15-17ByMP_Bigrams.csv')

In [184]:
session_names = session_df['MPs'].unique()

In [185]:
names2_not_in_session_names = [n for n in names2 if n not in session_names]
session_names_not_in_names2 = [n for n in session_names if n not in names2]

In [186]:
print(len(names2_not_in_session_names))
print(len(session_names_not_in_names2))

11
1


In [187]:
session_names_not_in_names2

['Laurence Robertson']

In [139]:
len(df1.loc[df1.Topic == 'European Union'])

6972

In [140]:
# df1 = pd.read_csv('2015_commons.csv')
# df2 = pd.read_csv('2016_commons.csv')
# df = pd.concat([df1, df2])
# df['len'] = df['Speech'].apply(lambda x: len(x.lower().split()))
# df1 = pd.read_csv('2016_commons.csv')
# df2 = pd.read_csv('2017_commons.csv')
# df1 = pd.concat([df1, df2])
# df1['len'] = df1['Speech'].apply(lambda x: len(x.lower().split()))

In [141]:
df

,Date,Speaker,Name,Party,Government,Topic,Topic-Stemmed,Speech-Stemmed-unigram_count,Speech-Stemmed-bigram_count,Speech-Stemmed-trigram_count,len
18954,2015-06-01,Chris Bryant (Rhondda) (Lab),Chris Bryant,Labour,No,Others,f.i.f.a.,urgent question ask make statement action gove...,urgent_question question_ask ask_make make_sta...,urgent_question_ask question_ask_make ask_make...,13
18955,2015-06-01,"The Secretary of State for Culture, Media and ...",John Whittingdale,Conservative,Yes,Others,f.i.f.a.,last friday f.i.f.a. member opportun embrac ov...,last_friday friday_f.i.f.a. f.i.f.a._member me...,last_friday_f.i.f.a. friday_f.i.f.a._member f....,161
18956,2015-06-01,Chris Bryant,Chris Bryant,Labour,No,Others,f.i.f.a.,sepp blatter shown bring reform f.i.f.a. may s...,sepp_blatter blatter_shown shown_bring bring_r...,sepp_blatter_shown blatter_shown_bring shown_b...,148
18957,2015-06-01,Mr Whittingdale,John Whittingdale,Conservative,Yes,Others,f.i.f.a.,agre comment sepp complet one need go soon rai...,agre_comment comment_sepp complet_one one_need...,agre_comment_sepp complet_one_need one_need_go...,172
18958,2015-06-01,Mr Speaker,Mr Speaker,Not Found,No,Others,f.i.f.a.,understand interest subject urgenc reflect sel...,understand_interest interest_subject subject_u...,understand_interest_subject interest_subject_u...,26
...,...,...,...,...,...,...,...,...,...,...,...
36254,2016-06-30,Albert Owen (Ynys Môn) (Lab),Albert Owen,Labour,No,Others,market town centr regener,know attend previou agre other spoke import hi...,know_attend attend_previou agre_other other_sp...,know_attend_previou agre_other_spoke other_spo...,23
36255,2016-06-30,Michelle Donelan,Michelle Donelan,Conservative,No,Others,market town centr regener,complet corsham suffer similar bank branch los...,corsham_suffer suffer_similar similar_bank ban...,corsham_suffer_similar suffer_similar_bank sim...,246
36256,2016-06-30,Chris Davies (Brecon and Radnorshire) (Con),Chris Davies,Conservative,No,Others,market town centr regener,may make clear devolv nation need includ one s...,may_make make_clear clear_devolv devolv_nation...,may_make_clear make_clear_devolv clear_devolv_...,22
36257,2016-06-30,Michelle Donelan,Michelle Donelan,Conservative,No,Others,market town centr regener,thank intervent quit agre strategi divers take...,thank_intervent intervent_quit quit_agre agre_...,thank_intervent_quit intervent_quit_agre quit_...,819


In [142]:
df['Topic'].unique()

array(['Others', 'Defence', 'European Union', 'Education',
       'Health services and medicine', 'Energy and environment',
       'International affairs', 'Housing and planning',
       'Economy and finance', 'Parliament, government and politics',
       'Asylum, immigration and nationality',
       'Social security and pensions', 'Transport',
       'Communities and families', 'Crime, civil law, justice and rights',
       'Business, industry and consumers', 'Employment and training',
       'Culture, media and sport', 'Science and technology',
       'Agriculture, animals, food and rural affairs', 'Social services'],
      dtype=object)

In [143]:
# df['Name'].unique()

In [144]:
df_eu = df.loc[df['Topic'] == 'European Union']
df1_eu = df1.loc[df1['Topic'] == 'European Union']

In [160]:
rows = []
for p in politicians:
    row = [p, 0, 0, 0, 0]
    
    if p not in df['Name'].values and p not in df1['Name'].values:
        pass
    if p in df['Name'].values:
        row[1] = df.loc[df['Name'] == p]['len'].sum()
    if p in df_eu['Name'].values:
        row[2] = df_eu.loc[df_eu['Name'] == p]['len'].sum()
    if p in df1['Name'].values:
        row[3] = df1.loc[df1['Name'] == p]['len'].sum()
    if p in df1_eu['Name'].values:
        row[4] = df1_eu.loc[df1_eu['Name'] == p]['len'].sum()
    rows.append(row)

cols = ['Politician', '2015-2016', '2015-2016 EU', '2016-2017', '2016-2017 EU']
res_df = pd.DataFrame(rows, columns=cols)
res_df

,Politician,2015-2016,2015-2016 EU,2016-2017,2016-2017 EU
0,David Lammy,12560,33,7530,822
1,Helen Goodman,8970,352,9537,3036
2,George Eustice,4152,49,8892,1252
3,David Rutley,7496,511,3729,511
4,Sarah Olney,0,0,2644,426
...,...,...,...,...,...
655,Chris Bryant,14263,75,8131,1020
656,Ann Clwyd,2945,376,4463,442
657,Julian Smith,0,0,0,0
658,Sarah Wollaston,9348,902,5953,409


In [146]:
res_df.to_csv('politicians_speech_indicator_with_word_count.csv')

In [147]:
brexit_file['2015-2016'] = brexit_file['full name'].apply(lambda x: df.loc[df['Name'] == x]['len'].sum())
brexit_file['2015-2016 EU'] = brexit_file['full name'].apply(lambda x: df_eu.loc[df_eu['Name'] == x]['len'].sum())
brexit_file['2016-2017'] = brexit_file['full name'].apply(lambda x: df1.loc[df1['Name'] == x]['len'].sum())
brexit_file['2016-2017 EU'] = brexit_file['full name'].apply(lambda x: df1_eu.loc[df1_eu['Name'] == x]['len'].sum())

In [148]:
brexit_file.to_csv('brexit_full_updated3_with_politicians_indicator_with_word_count.csv')

In [149]:
brexit_file

,Constituency ID,full name,PANO,Constituency Name,Party abbreviation,conShare,conShare2,conShare3,bill161,leave,party_old,ref,ref_dummy,party,old_party,2015-2016,2015-2016 EU,2016-2017,2016-2017 EU
0,E14000530,Gerald Howarth,7,Aldershot,Con,50.59,73.403947,73.403947,1.0,1.0,1.0,0.578978,1,1.0,1.0,10984,773,3280,698
1,E14000531,Wendy Morton,8,Aldridge-Brownhills,Con,52.05,69.940876,69.940876,1.0,0.0,1.0,0.677963,1,0.0,0.0,6952,201,11025,662
2,E14000532,Graham Brady,9,Altrincham and Sale West,Con,52.99,66.503514,66.503514,1.0,1.0,1.0,0.385878,0,1.0,1.0,2852,0,628,0
3,E14000533,Nigel Mills,11,Amber Valley,Con,43.98,55.840528,55.840528,1.0,1.0,1.0,0.652991,1,1.0,1.0,4477,0,5798,104
4,E14000534,Nick Herbert,18,Arundel and South Downs,Con,60.79,84.442284,84.442284,NaN,0.0,1.0,0.497011,0,0.0,0.0,3577,317,2819,87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,W07000076,Wayne David,114,Caerphilly,Lab,16.59,27.223499,27.223499,1.0,0.0,0.0,0.551360,1,0.0,NaN,7053,454,3326,525
656,W07000077,Chris Evans,336,Islwyn,Lab,15.16,23.639482,23.639482,1.0,0.0,0.0,0.589399,1,0.0,NaN,4697,0,5222,337
657,W07000078,Alun Cairns,589,Vale Of Glamorgan,Con,46.02,58.527280,58.527280,1.0,0.0,1.0,0.525507,1,0.0,0.0,8127,0,12034,0
658,W07000079,Kevin Brennan,129,Cardiff West,Lab,25.15,38.221884,38.221884,0.0,0.0,0.0,0.438226,0,0.0,NaN,15296,71,7025,123


KeyError: 'Sarah Champion'